# Selection and analysis of a deep learning model using Keras

In [1]:
#Libraries required for this section:

import numpy as np
from skimage.transform import resize
from keras import models
from keras import layers


C:\Users\JORGE\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## i) Selection:

The problem of counting number of object instances visible in a single image has been studied in depth by a wide variety of algorithm-based methodologies which can be classified, according to ['Crowd counting and proﬁling: Methodology and evaluation[1]'](#ref), in three groups: counting by clustering, counting by detection and counting by regresion. 

We have selected a 'counting by regresion' model because they have demonstrated to be the most accuareate and fastest ones, setting the state-of-the-art results in most of the benchmarks so far.

The selected model is based on the ['Towards perspective-free object counting with deep learning'](http://agamenon.tsc.uah.es/Investigacion/gram/publications/eccv2016-onoro.pdf) paper by Daniel Oñoro-Rubio and Roberto J. López-Sastre [[2]](#ref).

This paper was chosen because:
- It is supported by extensive and detailed documentation such as a GitHub project that includes trained models,
- The models subject to analysis in this paper are simpler, more compact and less computational expensive than other similar ones such as ['the Crowd_CNN model by Zhang et al. [3]'](#ref).
- A priori, one of the data sets used by this paper for experimental purpuses, TRANCOS, could have an interesting practical application to traffic density estimation. 
- It claims that their models outperform 'state-of-the-art' results as of 2016.

This paper develops 2 deep learning models based on CNN, Counting_CNN and Hydra_CNN, for objects density map estimation.<br>

I will use the '3-head' Hydra_CNN architecture since, in accordance with the paper, it is the model that provides the best results in terms of metrics.



## ii) Analysis:
"3-head Hydra model" (based on a 3-head Hydra_CNN**) phases:<br>
1. Data preprocessing: original image breakdown into 115x115 overlapped patches 
2. 3-head Hydra_CNN processing: Obtaining densitiy map estimation for each 115x115 patch **(MODEL TRANSLATION TO KERAS inclusive)**
3. Density map assembly: Assembling all the estimated maps to get the density map for the whole original image.

**Although the model includes a 3-head Hydra_CNN, the CNN is just a part (the key part) of the model.
That´s why we will refer to the whole model as '3-head Hydra model' (excluding the word 'CNN' from the model´s name).


### 1) Data Preprocessing:

Steps:<br>
1.1) Extract from the original image all the consecutive 115x115 patches with a stride of 10 pixels. <br>
1.2) Build a 'pyramid' of 3 different scale levels ('s') from each extracted patch: <br>
         - s_0: The original patch.
         - s_1: Centered crop of 66% of the original patch.
         - s_2: Centered crop of 33% of the original patch.
1.3) Resize s_0, s_1 and s_2 to 72x72 pixels to feed the Hydra_CNN.


In [2]:
#The following functions will be used in this phase:

In [3]:
def cartesian(arrays, out=None):
    """
    To be used in 1.1)
    Generates a cartesian product of input arrays.
    This function will provide a list of coordinates (positions) required by 'get_dense_pos' function (see below)
    to select those that will be used to build the patches (115x115).
    
    """    
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = int(n / arrays[0].size)
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
            
    return out

In [4]:
def get_dense_pos(heith, width, pw, stride = 1):
    
    '''
    To be used in 1.1)
    
    The patches will be created by selecting points of the image as the center of the patch.
    Thus, the points of the image that generate patches whose coordinates go beyond the image´s perimeter 
    will not be selected.
    
    This function provides a dense list of points that will be used to build the patches (115x115).
    
    Those points will be selected from the area of the image whose points are at a distance from image´s perimeter 
    equal or higher than half of the patch height (or width), with a stride set by the 'stride' parameter 
    (stride=10 in Hydra model).
    
    @param heith: image height.
    @param width: image width.
    @param pw: patch with.
    @param stride: stride.
  
    '''    
    # Computes half a patch (height or width, it´s the same)
    dx=dy=int(pw/2)
    
    # Coordinates of the area of the image whose points are at a distance from its perimeter equal or higher
    #than half of a patch: 
    pos = cartesian( (range(dx, heith - dx, stride), range(dy, width -dy, stride) ) )
    bot_line = cartesian( (heith - dx -1, range(dy, width -dy, stride) ) )
    right_line = cartesian( (range(dx, heith - dx, stride), width -dy - 1) )
    
    return np.vstack( (pos, bot_line, right_line) )

In [5]:
def extractEscales(lim, n_scales, input_pw):
    '''  
    To be used in 1.2) and 1.3)
    
    Builds a 'pyramid' of different scale levels for each extracted patch, and
    resizes the crops (s_0, s_1 and s_2 in our case) to the input size required to feed the Hydra_CNN.

    @param lim: list of patches related to the original image
    @param n_scales: number of different scale levels (3 in our case)
    @param input_pw: input size (72x72 pixels)
    
    '''
    out_list = [] #List of 'pyramids' corresponding to all the patches extracted from the original image
    for im in lim:
        ph, pw = im.shape[0:2] # it will get the patch width and height (115x115)
        scaled_im_list = [] #list of crops ('pyramid') related to a specific patch
        
        #Crops generating and resizing:
        for s in range(n_scales):
            ch = int(s * ph / (2*n_scales)) 
            cw = int(s * pw / (2*n_scales))
            
            crop_im = im[ch:ph-ch, cw:pw - cw]
            
            #s=0, s_0 = original patch = (115x115)
            #s=1, s_1 = 66% of original patch = (77x77)
            #s=2, s_2 = 33% of original patch = (39x39)
            
            #Resizes the crops (s_0, s_1 and s_2 in our case) to the input size required to feed the Hydra_CNN (72x72)
            scaled_im_list.append(resize(crop_im, (input_pw, input_pw)))
        
        out_list.append(scaled_im_list)
        
    return out_list

## 2) 3-head Hydra_CNN processing: Obtaining densitiy map estimation for each 115x115 patch 

Hydra_CNN works learning a multiscale non-linear regressor from the input image features to obtain an object density map.<br>
Therefore, each preprocessed **115x115 patch** extracted from the original image **('pyramid'** made up of **s_0, s_1 and s_2 crops resized to 72x72)** goes through the Hydra_CNN to obtain their **density map**.

### 2.1. "3-head Hydra_CNN" Architecture:

**- Input:**<br>
The network is fed with 'pyramids' made up of **3 input crops (s_0, s_1 and s_2)**, extracted from each 115x115 patch and rescaled to a size of 72x72.

**- Heads:**<br>
Each head will learn the paterns for a particular scale level from the input 'pyramid'.<br>
Therefore, **there are 3 heads**.<br>

All the heads have the same architecture, consisting of **5 convolutional layers**:<br>
**Conv1** and **Conv2** layers have filters of size 7x7 with a depth of 32, and they are followed by a max-pooling layer, with 2x2 kernel size.<br>
The **Conv3** layer has 5x5 filters with a depth of 64, followed by a max-pooling layer, with 2x2 kernel size.<br>
**Conv4** y **Conv5** layers are made up of 1x1 filters with a depth of 1000 and 400, respectively.<br>
All these convolutional layers are followed by rectified linear units (ReLU).

![title](./Notebook_pics/CNN_layers.png)
**Fig.1.**  *Heads structure: consisting of 5 covolutional layers (figure extracted from the paper ['Towards perspective-free object counting with deep learning'](http://agamenon.tsc.uah.es/Investigacion/gram/publications/eccv2016-onoro.pdf))*.

**- Body**:<br>
The output of the heads is a set of features describing the images at different scales.<br> 
These features are concatedated to feed the 'body', which is made up of fully-connected layers.<br>
There are 3 layers:<br>

**Fc6** and **Fc7** have 512 neurons followed by a ReLU and a dropout layer, each one.<br>
**Fc8** is the final layer, with 324 neurons, whose output is the object density map.<br>

**Note: The format of the density map returned by this model is a vector with a dimension of 324.**


![title](./Notebook_pics/hydra_architecture.png)
**Fig.2**. *n-head Hydra_CNN Architecture (figure extracted from the paper ['Towards perspective-free object counting with deep learning'](http://agamenon.tsc.uah.es/Investigacion/gram/publications/eccv2016-onoro.pdf))*.<br>



### 2.2. Translation to Keras:
The original model uses the Caffe library so, **an important contribution of this project** is the translation of the whole model from Caffe into Keras.

Note: In this section, we will not include any specific element for training purposes. 

The translation is as follows:

In [6]:
def hydra_CNN():

### CCNN_s0:
    #Input:
    head0_input = layers.Input(shape=(3,72,72))
    #Conv1:
    head0 = layers.ZeroPadding2D(padding=(3, 3), data_format="channels_first")(head0_input)
    head0 = layers.Conv2D(32, kernel_size=7, activation='relu', name="head0_conv1", data_format="channels_first")(head0)
    head0 = layers.MaxPooling2D(pool_size=2, strides=2, data_format="channels_first")(head0)
    #Conv2:
    head0 = layers.ZeroPadding2D(padding=(3, 3), data_format="channels_first")(head0)
    head0 = layers.Conv2D(32, kernel_size=7, activation='relu', name="head0_conv2", data_format="channels_first")(head0)
    head0 = layers.MaxPooling2D(pool_size=2, strides=2, data_format="channels_first")(head0)
    #Conv3:
    head0 = layers.ZeroPadding2D(padding=(2, 2), data_format="channels_first")(head0)
    head0 = layers.Conv2D(64, kernel_size=5, activation='relu', name="head0_conv3", data_format="channels_first")(head0)
    #Conv4:
    head0 = layers.Conv2D(1000, kernel_size=1, activation='relu', name="head0_conv4", data_format="channels_first")(head0)
    #Conv5:
    head0 = layers.Conv2D(400, kernel_size=1, activation='relu', name="head0_conv5", data_format="channels_first")(head0)


### CCNN_s1:
    #Input:
    head1_input = layers.Input(shape=(3,72,72))
    #Conv1:
    head1 = layers.ZeroPadding2D(padding=(3, 3), data_format="channels_first")(head1_input)
    head1 = layers.Conv2D(32, kernel_size=7, activation='relu', name="head1_conv1", data_format="channels_first")(head1)
    head1 = layers.MaxPooling2D(pool_size=2, strides=2, data_format="channels_first")(head1)
    #Conv2:
    head1 = layers.ZeroPadding2D(padding=(3, 3), data_format="channels_first")(head1)
    head1 = layers.Conv2D(32, kernel_size=7, activation='relu', name="head1_conv2", data_format="channels_first")(head1)
    head1 = layers.MaxPooling2D(pool_size=2, strides=2, data_format="channels_first")(head1)
    #Conv3:
    head1 = layers.ZeroPadding2D(padding=(2, 2), data_format="channels_first")(head1)
    head1 = layers.Conv2D(64, kernel_size=5, activation='relu', name="head1_conv3", data_format="channels_first")(head1)
    #Conv4:
    head1 = layers.Conv2D(1000, kernel_size=1, activation='relu', name="head1_conv4", data_format="channels_first")(head1)
    #Conv5:
    head1 = layers.Conv2D(400, kernel_size=1, activation='relu', name="head1_conv5", data_format="channels_first")(head1)


### CCNN_s2:
    #Input:
    head2_input = layers.Input(shape=(3,72,72))
    #Conv1:
    head2 = layers.ZeroPadding2D(padding=(3, 3), data_format="channels_first")(head2_input)
    head2 = layers.Conv2D(32, kernel_size=7, activation='relu', name="head2_conv1", data_format="channels_first")(head2)
    head2 = layers.MaxPooling2D(pool_size=2, strides=2, data_format="channels_first")(head2)
    #Conv2:
    head2 = layers.ZeroPadding2D(padding=(3, 3), data_format="channels_first")(head2)
    head2 = layers.Conv2D(32, kernel_size=7, activation='relu', name="head2_conv2", data_format="channels_first")(head2)
    head2 = layers.MaxPooling2D(pool_size=2, strides=2, data_format="channels_first")(head2)
    #Conv3:
    head2 = layers.ZeroPadding2D(padding=(2, 2), data_format="channels_first")(head2)
    head2 = layers.Conv2D(64, kernel_size=5, activation='relu', name="head2_conv3", data_format="channels_first")(head2)
    #Conv4:
    head2 = layers.Conv2D(1000, kernel_size=1, activation='relu', name="head2_conv4", data_format="channels_first")(head2)
    #Conv5:
    head2 = layers.Conv2D(400, kernel_size=1, activation='relu', name="head2_conv5", data_format="channels_first")(head2)

#Note: Caffe uses a "channel_first" format for inputs.

## Concatenation of outputs from the CCNN layers (s0, s1 and s2):
    body = layers.concatenate([head0, head1, head2], axis=1)
    body = layers.Flatten()(body)
#Note: A 'Flatten' layer is added, which it´s not mentioned in the paper, 
#to be able to feed the fully-connected layers properly. 


## Fully-connected layers:
    #Fc6:
    body = layers.Dense(512, activation='relu', name="body_fc6")(body)
    #Fc7:
    body = layers.Dense(512, activation='relu', name="body_fc7")(body)
    #Fc8:
    body = layers.Dense(324, activation='relu', name="body_fc8")(body)


## Model definition:
    hydra_CNN_model = models.Model(inputs=[head0_input, head1_input, head2_input], outputs=body)

    return hydra_CNN_model


Let´s have a look at an instance of a '3-head Hydra_CNN'

In [7]:
hydra_sample = hydra_CNN()
hydra_sample.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 72, 72)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 3, 72, 72)    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 3, 72, 72)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 3, 78, 78)    0           input_1[0][0]                    
__________________________________________________________________________________________________
zero_paddi

## 3) Density map assembly: Assembling all the estimated maps to get the density map for the whole original image.

Steps:<br>

### 3.1) Reshape&resize Hydra_CNN´s outputs to get the density map for each patch:
As mentioned before, the **output** of the Hydra_CNN is a **1x324 vector**.<br>
In order to get the density map of each patch, their corresponding vector will be **reshaped to a 2-dimention array**.<br>
Due to the max-pooling layers, the size of **density map estimations is 1/4 of the input image patch (18x18 pixels)**.<br>
Thus, the reshaped output will have to be **resized** to the original size of the patch (115x115).<br>

**The following function will be used for this purpose:**

In [8]:
def resizeDensityPatch(patch, opt_size):
    '''
    Takes a density map and resizes it to the opt_size.
    
    @param patch: input density map.
    @param opt_size: output size (the original size of the patch, 115x115, in our case).
        
    The rescaling process will generate a density map whose associated count (pixel values addition) will not
    necessarily match the count of the input density map.
    Therefore, the input density map will be normalized before rescaling, then reversed after rescaling
    and the final count adjusted.
    
    '''
       
    # Input normalization to values between 0 and 1:
    patch_sum = patch.sum()   
    p_max = patch.max()
    p_min = patch.min()    
    if patch_sum !=0: # Avoid 0 division
        patch = (patch - p_min)/(p_max - p_min)
    
    # Resizing to the original size of the patch:
    patch = resize(patch, opt_size)
    
    # Normalization reversal:
    patch = patch*(p_max - p_min) + p_min
    
    # Count adjustment:
    res_sum = patch.sum()
    if res_sum != 0:
        return patch * (patch_sum/res_sum)

    return patch

### 3.2) Assembly of all density map estimations:
**All** the **density map estimations** for the extracted patchs are **aggregated on a single density** map with the size
of the original image.<br>
Since **patches overlap**, the times each coordinate of the final density map is estimated (votes) will be calculated and used to get the **average estimation** for each coordinate.

**The final function required to run the model is as follows:**

In [9]:
#Hydra model processing:

def process(model, im, n_scales, base_pw, input_pw):
    
    '''
    Parameters (values based on the paper´s methodology):
    @param model: model to be used for density map estimation purposes (3-head Hydra_CNN)
    @param im: image from which a density map will be estimated
    @param n_scales: number of different scale levels to feed the Hydra_CNN as inputs (for the 3-head Hydra_CNN, 
                there will be 3: s:0, s_1, s_2)
    @param base_pw: original size of patches to be extracted from the image (115x115 pixels)
    @param inpu_pw: inputs size (72x72 pixels)
    
    '''
#1) Data preprocessing: original image breakdown into 115x115 overlapped patches
    #Steps:
    
    #1.1) Extract from the original image all the consecutive 115x115 patches with a stride of 10 pixels: 
            
    # Obtaining a dense list of points (coordinates) from the image that will be used to build the patches (115x115)
    [heith, width] = im.shape[0:2]
    pos = get_dense_pos(heith, width, base_pw, stride=10) #Stride=10

    # Initialize density matrix and votes counting
    dens_map = np.zeros( (heith, width), dtype = np.float32 )  
    count_map = np.zeros( (heith, width), dtype = np.int32 )  
                
    for ix, p in enumerate(pos): # Iterate for all patches            
        dx=dy=int(base_pw/2) # Compute displacement from centers
        x,y=p
        sx=slice(x-dx, x+dx+1, None)
        sy=slice(y-dy, y+dy+1, None)
        crop_im=im[sx,sy,...]
        h, w = crop_im.shape[0:2]
        if h!=w or (h<=0):
            continue
    #1.2) Build a 'pyramid' of 3 different scale levels ('s') for each extracted patch, and,
    #1.3) Resize s_0, s_1 and s_2 to 72x72 pixels to feed the Hydra_CNN.
           
        im_scales = extractEscales([crop_im], n_scales, input_pw)
                        
        head0_input = np.expand_dims(im_scales[0][0].copy().transpose(2,0,1), axis=0)
        head1_input = np.expand_dims(im_scales[0][1].copy().transpose(2,0,1), axis=0)
        head2_input = np.expand_dims(im_scales[0][2].copy().transpose(2,0,1), axis=0)
            
#2) 3-head Hydra_CNN processing: Obtaining densitiy map estimation for each 115x115 patch         
        pred = model.predict([head0_input,  head1_input,  head2_input])

#3) Density map assembly: Assembling all the estimated maps to get the density map for the whole original image

    #3.1) Reshape&resize Hydra_CNN´s outputs to get the density map for each patch:
        #Rashape to a 2-dimention array
        p_side = int(np.sqrt( len( pred.flatten() ) )) 
        pred = pred.reshape(  (p_side, p_side) )
            
            # Resize it back to the original patch size (115x115 pixels)
        pred = resizeDensityPatch(pred, crop_im.shape[0:2])          
        pred[pred<0] = 0

    #3.2) Assembly of all patches density map estimations:
        # The estimated density map for each patch is added to the total density map:
        dens_map[sx,sy] += pred
        #Since overlapping occurs, a matrix is summing up the times (votes) each coordinate has been estimated, 
        #for afterwards average calculation purposes:
        count_map[sx,sy] += 1

    # Remove Zeros
    count_map[ count_map == 0 ] = 1

    # Final average density map for the whole original image:
    dens_map = dens_map / count_map        
        
    return dens_map

<a id="ref"></a>
### References:
[1]: Loy, C., Chen, K., Gong, S., Xiang, T.: Crowd counting and proﬁling: Methodology and evaluation. In: Modeling, Simulation and Visual Analysis of Crowds. (2013) <br>

[2]: Oñoro-Rubio D., López-Sastre R.J. (2016) Towards Perspective-Free Object Counting with Deep Learning. In: Leibe B., Matas J., Sebe N., Welling M. (eds) Computer Vision – ECCV 2016. ECCV 2016. Lecture Notes in Computer Science, vol 9911. Springer, Cham <br>

[3]: Zhang, C., Li, H., Wang, X., Yang, X.: Cross-scene crowd counting via deep convolutional neural networks. In: CVPR. (June 2015) <br>
